Let's make sure that we have access to GPU. We can use `nvidia-smi` command to do that. In case of any problems navigate to `Edit` -> `Notebook settings` -> `Hardware accelerator` and set it to `GPU`.

In [20]:
!nvidia-smi


Tue Jul  2 09:05:24 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.85                 Driver Version: 555.85         CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3060 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   47C    P8             16W /   40W |    1110MiB /   6144MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Install Detectron2 and dependencies

In [21]:
!pip install git+https://github.com/facebookresearch/detectron2.git

  Cloning https://github.com/facebookresearch/detectron2.git to c:\users\daanish mittal\appdata\local\temp\pip-req-build-44wg59de
    Complete output from command python setup.py egg_info:
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "C:\Users\Daanish Mittal\AppData\Local\Temp\pip-req-build-44wg59de\setup.py", line 10, in <module>
        import torch
    ModuleNotFoundError: No module named 'torch'
    
    ----------------------------------------


Command "python setup.py egg_info" failed with error code 1 in C:\Users\Daanish Mittal\AppData\Local\Temp\pip-req-build-44wg59de\
You are using pip version 18.1, however version 21.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


Now is a good time to confirm that we have the right versions of the libraries at our disposal.

In [22]:
!pip install -U "iopath<0.1.10,>=0.1.7"
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
import pkg_resources
print("detectron2:", pkg_resources.get_distribution("detectron2").version)

Requirement already up-to-date: iopath<0.1.10,>=0.1.7 in c:\users\daanish mittal\appdata\local\programs\python\python36\lib\site-packages (0.1.9)


You are using pip version 18.1, however version 21.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Fri_Feb__8_19:08:26_Pacific_Standard_Time_2019
Cuda compilation tools, release 10.1, V10.1.105
torch:  2.0 ; cuda:  cu117
detectron2: 0.6


In [23]:
# COMMON LIBRARIES
import os
import cv2

from datetime import datetime
import cv2

# DATA SET PREPARATION AND LOADING
from detectron2.data.datasets import register_coco_instances
from detectron2.data import DatasetCatalog, MetadataCatalog

# VISUALIZATION
from detectron2.utils.visualizer import Visualizer
from detectron2.utils.visualizer import ColorMode

# CONFIGURATION
from detectron2 import model_zoo
from detectron2.config import get_cfg

# EVALUATION
from detectron2.engine import DefaultPredictor

# TRAINING
from detectron2.engine import DefaultTrainer

## Run a Pre-trained Detectron2 Model

In [24]:
# !wget http://images.cocodataset.org/val2017/000000439715.jpg -q -O input.jpg
# image = cv2.imread("./input.jpg")
# cv2.imshow('Display Window', image)  # 'Display Window' is the name of the window
# cv2.waitKey(0)  # Waits for a key press to close the window
# cv2.destroyAllWindows()  # Closes the window

In [25]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117 --upgrade --force-reinstall

Looking in indexes: https://download.pytorch.org/whl/cu117


  Could not find a version that satisfies the requirement torch (from versions: )
No matching distribution found for torch
You are using pip version 18.1, however version 21.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


## COCO Format Dataset

In [26]:
!pip install roboflow

  Using cached https://files.pythonhosted.org/packages/99/f2/8349482b1cb7d408cd8673efaa1b51e7bd657f95b679de27f89e333f1388/roboflow-1.1.6-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/3f/51/d4db610ef29373b879047326cbf6fa98b6c1969d6f6dc423279de2b1be2c/requests_toolbelt-1.0.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/2d/61/08076519c80041bc0ffa1a8af0cbd3bf3e2b62af10435d269a9d0f40564d/requests-2.27.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ae/6a/99eaaeae8becaa17a29aeb334a18e5d582d873b6f084c11f02581b8d7f7f/urllib3-1.26.19-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/a2/38/928ddce2273eaa564f6f50de919327bf3a00f091b5baba8dfa9460f3a8a8/idna-2.10-py2.py3-none-any.whl


  Could not find a version that satisfies the requirement supervision (from roboflow) (from versions: )
No matching distribution found for supervision (from roboflow)
You are using pip version 18.1, however version 21.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [27]:

from roboflow import Roboflow
rf = Roboflow(api_key="RovqaIFPpcekpUVWjRke")
project = rf.workspace("tank-5yib6").project("tank-and-ramp-1")
version = project.version(1)
dataset = version.download("coco-segmentation")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to tank-and-ramp-1-1 in coco-segmentation:: 100%|██████████| 354/354 [00:00<00:00, 1351.88it/s]


### Register

When you use Detectron2, before you actually train the model you need to [register it](https://detectron2.readthedocs.io/en/latest/tutorials/datasets.html#register-a-coco-format-dataset).

In [28]:
DATA_SET_NAME = dataset.name.replace(" ", "-")
ANNOTATIONS_FILE_NAME = "_annotations.coco.json"

In [29]:
# TRAIN SET
TRAIN_DATA_SET_NAME = f"{DATA_SET_NAME}-train"
TRAIN_DATA_SET_IMAGES_DIR_PATH = os.path.join(dataset.location, "train")
TRAIN_DATA_SET_ANN_FILE_PATH = os.path.join(dataset.location, "train", ANNOTATIONS_FILE_NAME)

register_coco_instances(
    name=TRAIN_DATA_SET_NAME,
    metadata={},
    json_file=TRAIN_DATA_SET_ANN_FILE_PATH,
    image_root=TRAIN_DATA_SET_IMAGES_DIR_PATH
)

# TEST SET
TEST_DATA_SET_NAME = f"{DATA_SET_NAME}-test"
TEST_DATA_SET_IMAGES_DIR_PATH = os.path.join(dataset.location, "test")
TEST_DATA_SET_ANN_FILE_PATH = os.path.join(dataset.location, "test", ANNOTATIONS_FILE_NAME)

register_coco_instances(
    name=TEST_DATA_SET_NAME,
    metadata={},
    json_file=TEST_DATA_SET_ANN_FILE_PATH,
    image_root=TEST_DATA_SET_IMAGES_DIR_PATH
)

# VALID SET
VALID_DATA_SET_NAME = f"{DATA_SET_NAME}-valid"
VALID_DATA_SET_IMAGES_DIR_PATH = os.path.join(dataset.location, "valid")
VALID_DATA_SET_ANN_FILE_PATH = os.path.join(dataset.location, "valid", ANNOTATIONS_FILE_NAME)

register_coco_instances(
    name=VALID_DATA_SET_NAME,
    metadata={},
    json_file=VALID_DATA_SET_ANN_FILE_PATH,
    image_root=VALID_DATA_SET_IMAGES_DIR_PATH
)

We can now confirm that our custom dataset was correctly registered using [MetadataCatalog](https://detectron2.readthedocs.io/en/latest/modules/data.html#detectron2.data.MetadataCatalog).

In [30]:
[
    data_set
    for data_set
    in MetadataCatalog.list()
    if data_set.startswith(DATA_SET_NAME)
]

['tank-and-ramp-1-train', 'tank-and-ramp-1-test', 'tank-and-ramp-1-valid']

### Visualize

Let's take a look at single entry from out train dataset.

In [31]:
metadata = MetadataCatalog.get(TRAIN_DATA_SET_NAME)
dataset_train = DatasetCatalog.get(TRAIN_DATA_SET_NAME)

dataset_entry = dataset_train[0]
image = cv2.imread(dataset_entry["file_name"])

visualizer = Visualizer(
    image[:, :, ::-1],
    metadata=metadata,
    scale=0.5,
    instance_mode=ColorMode.IMAGE_BW
)

out = visualizer.draw_dataset_dict(dataset_entry)
cv2.imshow("visualize",out.get_image()[:, :, ::-1])
cv2.waitKey(0)  # Waits for a key press to close the window
cv2.destroyAllWindows()  # Closes the window


Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.



## Train Model Using Custom COCO Format Dataset

### Configuration

In [32]:
# HYPERPARAMETERS
ARCHITECTURE = "mask_rcnn_R_101_FPN_3x"
CONFIG_FILE_PATH = f"COCO-InstanceSegmentation/{ARCHITECTURE}.yaml"
MAX_ITER = 2000
EVAL_PERIOD = 200
BASE_LR = 0.001
NUM_CLASSES = 3

# OUTPUT DIRa
OUTPUT_DIR_PATH = os.path.join(
    DATA_SET_NAME,
    ARCHITECTURE,
    datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
)

os.makedirs(OUTPUT_DIR_PATH, exist_ok=True)

In [33]:
# cfg = get_cfg()
# cfg.merge_from_file(model_zoo.get_config_file(CONFIG_FILE_PATH))
# cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(CONFIG_FILE_PATH)
# cfg.DATASETS.TRAIN = (TRAIN_DATA_SET_NAME,)
# cfg.DATASETS.TEST = (TEST_DATA_SET_NAME,)
# cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64
# cfg.TEST.EVAL_PERIOD = EVAL_PERIOD
# cfg.DATALOADER.NUM_WORKERS = 2
# cfg.SOLVER.IMS_PER_BATCH = 2
# cfg.INPUT.MASK_FORMAT='bitmask'
# cfg.SOLVER.BASE_LR = BASE_LR
# cfg.SOLVER.MAX_ITER = MAX_ITER
# cfg.MODEL.ROI_HEADS.NUM_CLASSES = NUM_CLASSES
# cfg.OUTPUT_DIR = OUTPUT_DIR_PATH

In [34]:

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(CONFIG_FILE_PATH))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(CONFIG_FILE_PATH)
cfg.DATASETS.TRAIN = (TRAIN_DATA_SET_NAME,)
cfg.DATASETS.TEST = (TEST_DATA_SET_NAME,)
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512 # Default batch size per image

# Adjusting batch size and image size for GPU with 6GB VRAM
cfg.SOLVER.IMS_PER_BATCH = 3  # Overall batch size
cfg.INPUT.MIN_SIZE_TRAIN = (640, 672, 704, 736, 768, 800)  # Adjusted to fit memory constraints
cfg.INPUT.MIN_SIZE_TEST = 800
cfg.INPUT.MAX_SIZE_TRAIN = 1333
cfg.INPUT.MAX_SIZE_TEST = 1333

cfg.TEST.EVAL_PERIOD = EVAL_PERIOD
cfg.DATALOADER.NUM_WORKERS = 4
cfg.SOLVER.BASE_LR = BASE_LR
cfg.SOLVER.MAX_ITER = MAX_ITER
cfg.MODEL.ROI_HEADS.NUM_CLASSES = NUM_CLASSES
cfg.INPUT.MASK_FORMAT = 'bitmask'
cfg.OUTPUT_DIR = OUTPUT_DIR_PATH

# Save config for future reference
with open(os.path.join(OUTPUT_DIR_PATH, "config.yaml"), "w") as f:
    f.write(cfg.dump())

### Training

In [35]:
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[07/02 09:05:46 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (12, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (12,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (3, 256, 1, 1) 

[07/02 09:05:46 d2.engine.train_loop]: Starting training from iteration 0
[07/02 09:06:12 d2.utils.events]:  eta: 0:31:38  iter: 19  total_loss: 2.168  loss_cls: 1.283  loss_box_reg: 0.1989  loss_mask: 0.6855  loss_rpn_cls: 0.002302  loss_rpn_loc: 0.006246    time: 0.9589  last_time: 0.8227  data_time: 0.3073  last_data_time: 0.0049   lr: 1.9981e-05  max_mem: 4567M
[07/02 09:06:41 d2.utils.events]:  eta: 0:31:47  iter: 39  total_loss: 1.595  loss_cls: 0.679  loss_box_reg: 0.2476  loss_mask: 0.6495  loss_rpn_cls: 0.006761  loss_rpn_loc: 0.007813    time: 0.9890  last_time: 1.1177  data_time: 0.0030  last_data_time: 0.0037   lr: 3.9961e-05  max_mem: 4567M
[07/02 09:07:01 d2.utils.events]:  eta: 0:31:36  iter: 59  total_loss: 1.045  loss_cls: 0.2537  loss_box_reg: 0.1728  loss_mask: 0.5878  loss_rpn_cls: 0.002625  loss_rpn_loc: 0.006455    time: 0.9964  last_time: 1.0807  data_time: 0.0031  last_data_time: 0.0030   lr: 5.9941e-05  max_mem: 4580M
[07/02 09:07:21 d2.utils.events]:  eta: 0:3

In [36]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir $OUTPUT_DIR_PATH

### Evaluation

In [37]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7


predictor = DefaultPredictor(cfg)

[07/02 09:27:33 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from tank-and-ramp-1\mask_rcnn_R_101_FPN_3x\2024-07-02-09-05-45\model_final.pth ...


In [38]:
from detectron2.data import DatasetCatalog
import cv2
from detectron2.utils.visualizer import Visualizer, ColorMode

# Assuming VALID_DATA_SET_NAME and predictor are already defined
dataset_valid = DatasetCatalog.get(VALID_DATA_SET_NAME)

for d in dataset_valid:
    # Read the image
    img = cv2.imread(d["file_name"])
    
    # Get predictions
    outputs = predictor(img)
    
    # Visualize predictions
    visualizer = Visualizer(
        img[:, :, ::-1],  # Convert BGR to RGB
        metadata=metadata,  # Metadata for visualization
        scale=0.8,  # Scale of the image
        instance_mode=ColorMode.IMAGE_BW  # Display mode
    )
    out = visualizer.draw_instance_predictions(outputs["instances"].to("cpu"))
    
    # Show the image with predictions
    cv2.imshow("validation", out.get_image()[:, :, ::-1])  # Convert RGB back to BGR
    cv2.waitKey(0)  # Waits for a key press to close the window
    cv2.destroyAllWindows()  # Closes the window


WARNING [07/02 09:27:33 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[07/02 09:27:33 d2.data.datasets.coco]: Loaded 29 images in COCO format from c:\Users\Daanish Mittal\OneDrive\Desktop\Tank_align\tank_alignment\ramp_and_edge\tank-and-ramp-1-1\valid\_annotations.coco.json


In [39]:
import os
import cv2
import torch
from PIL import Image
from torchvision import transforms
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

def load_model(config_file, weights_file):
    # Load the configuration
    cfg = get_cfg()
    cfg.merge_from_file(config_file)
    cfg.MODEL.WEIGHTS = weights_file
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.96
      # Set a custom testing threshold
    return DefaultPredictor(cfg)

def process_image(predictor, image_path):
    # Load image
    image = cv2.imread(image_path)
    outputs = predictor(image)
    
    # Visualize the results
    v = Visualizer(image[:, :, ::-1], MetadataCatalog.get(predictor.cfg.DATASETS.TRAIN[0]), scale=1.2)
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    
    # Display the image
    result_image = out.get_image()[:, :, ::-1]
    cv2.imshow(f'Result for {image_path}', result_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def process_video(predictor, video_path):
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        outputs = predictor(frame)
        v = Visualizer(frame[:, :, ::-1], MetadataCatalog.get(predictor.cfg.DATASETS.TRAIN[0]), scale=1.2)
        out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
        
        # Display the frame
        result_frame = out.get_image()[:, :, ::-1]
        cv2.imshow(f'Result for a frame in {video_path}', result_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

def process_folder(predictor, folder_path):
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if os.path.isfile(file_path):
            if file_path.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff')):
                process_image(predictor, file_path)
            elif file_path.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
                process_video(predictor, file_path)

# Example usage
config_file = "C:/Users/Daanish Mittal/OneDrive/Desktop/Tank_align/tank_alignment/ramp_and_edge/tank-and-ramp-2/mask_rcnn_R_101_FPN_3x/2024-07-02-08-11-08/config.yaml"
weights_file = "C:/Users/Daanish Mittal/OneDrive/Desktop/Tank_align/tank_alignment/ramp_and_edge/tank-and-ramp-2/mask_rcnn_R_101_FPN_3x/2024-07-02-08-11-08/model_final.pth"
input_folder = "C:/Users/Daanish Mittal/OneDrive/Desktop/Tank_align/tank_alignment/ramp_and_edge/tank-and-ramp-2-1/test"

predictor = load_model(config_file, weights_file)
process_folder(predictor, input_folder)


AssertionError: Config file 'C:/Users/Daanish Mittal/OneDrive/Desktop/Tank_align/tank_alignment/ramp_and_edge/tank-and-ramp-2/mask_rcnn_R_101_FPN_3x/2024-07-02-08-11-08/config.yaml' does not exist!